In [6]:
from langchain.document_loaders import PyPDFLoader

In [7]:
%cd ..

d:\ds_projects\interview-question-creator


c:\Users\Abhinesh\anaconda3\envs\interview\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [25]:
file_path = "data\SDG.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [26]:
print(data[2])

page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from these countries created a plan called the Sustainable 
Development Goals (SDGs). This set of 17 goals imagines a future just 15 years 
off that would be rid of poverty and hunger, and safe from the worst effects of 
climate change. It’s an ambitious plan. 
But there’s ample evidence that we can succeed. In the past 15 ye

In [27]:
len(data)

24

In [28]:
question_gen = ""

for page in data:
    question_gen += page.page_content
print(question_gen)

IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from these countries created a plan called the Sustainable 
Development Goals (SDGs). This set of 17 goals imagines a future just 15 years 
off that would be rid of poverty and hunger, and safe from the worst effects of 
climate change. It’s an ambitious plan. 
But there’s ample evidence that we can succeed. In the past 15 years, the 
inte

## Creating chunks for Question

In [29]:
from langchain.text_splitter import TokenTextSplitter
splitter_ques_gen = TokenTextSplitter(
    chunk_size= 10000,
    chunk_overlap = 200
)

In [31]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)
len(chunk_ques_gen)

1

In [35]:
from langchain.docstore.document import Document

document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]
document_ques_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence 

## Creating chunks for answers

In [36]:
splitter_ans_gen = TokenTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

In [37]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [38]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence 

## Chain for Questions

In [39]:
from langchain.chat_models import ChatOllama

llm_ques_gen_pipeline = ChatOllama(
    model = 'llama3',
    temperature = 0.3
)

In [55]:
prompt_template = """
You are an expert at creating questions based on text material.
Your goal is to prepare a student for their exam.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the student for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [56]:
from langchain.prompts import PromptTemplate

In [57]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [58]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a student prepare for a test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
Only return a list of 10 questions and nothing else.
QUESTIONS:
"""
)

In [59]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [60]:
from langchain.chains.summarize import load_summarize_chain

In [61]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [62]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on text material.
Your goal is to prepare a student for their exam.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from these countries created a plan called the Sust

In [49]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3",
)

In [63]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(document_answer_gen, embeddings)
llm_answer_gen = ChatOllama(temperature=0.1, model="llama3")

In [64]:
ques

"Here are some questions based on the text material:\n\n**Marine and Coastal Diversity**\n\n1. How many people depend on marine and coastal diversity for their livelihoods?\n2. What is the current state of fish stocks, according to the text?\n3. Why are oceans important even for those who live nowhere near them?\n\n**Sustainable Development Goals (SDGs)**\n\n4. What are the SDGs' targets for managing and protecting life below water?\n5. How do humans rely on terrestrial ecosystems for food, clean air, and water?\n6. What is the goal of conserving and restoring terrestrial ecosystems by 2030?\n\n**Peaceful and Inclusive Societies**\n\n7. Why can't a country develop without peace?\n8. How does justice relate to peace and human rights?\n9. What are some ways to reduce all forms of violence, according to the SDGs?\n\n**Implementation and Global Partnership**\n\n10. What is the final goal of the Sustainable Development Goals?\n11. How many countries agreed on these goals?\n12. What is one w

In [69]:
ques_list = ques.split("\n")
ques_list = ques_list[4:7]
ques_list

['1. How many people depend on marine and coastal diversity for their livelihoods?',
 '2. What is the current state of fish stocks, according to the text?',
 '3. Why are oceans important even for those who live nowhere near them?']

In [70]:
from langchain.chains import RetrievalQA

answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())

In [71]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. How many people depend on marine and coastal diversity for their livelihoods?
Answer:  According to the United Nations Food and Agriculture Organization (FAO), approximately 1 billion people worldwide depend on marine and coastal ecosystems for their livelihoods. This includes:

* Small-scale fishers: around 200 million people
* Coastal communities that rely on fishing, aquaculture, or other marine-related activities: around 300-400 million people
* People involved in tourism, recreation, and other marine-based industries: around 100-200 million people
* Communities that depend on mangroves, coral reefs, estuaries, and other coastal ecosystems for food, shelter, and livelihoods: around 500-600 million people

These numbers are approximate, as the exact number of people dependent on marine and coastal diversity is difficult to quantify. However, it's clear that a significant portion of the global population relies on these ecosystems for their well-being and economic stabi